In [ ]:
# Magic methods that automatically update changes to imported python functions

%load_ext autoreload
%autoreload 2

In [1]:
year = 2019
url = f'https://api.census.gov/data/{year}/abscs?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F&for=us:*'

## Questions to Answer

1. Question 1 - Visual Type A : Time in business vs Wealth
2. Question 2 - Visual Type A : Size of business vs Wealth
3. Question 3 - Visual Type B : Employee Size vs Wealth

4. Question 4 - Visual Type A : Demographic of Employees
5. Question 5 - Visual Type A : Demographic of Employees
6. Question 6 - Visual Type B : Demographic of Employees

7. Question 7 - Visual Type A : Wealth by Business Type
8. Question 8 - Visual Type A : Wealth by Location
9. Question 9 - Visual Type B : Mystery Question


In [2]:
import requests
import pandas as pd

YearinBusinessDF = pd.read_csv('csv\ABSCS2018.AB1800CSA02-2023-04-20T164944.csv')
ReceiptSizeOfFirmDF = pd.read_csv('csv\ABSCS2018.AB1800CSA03-2023-04-20T164951.csv')
EmploymentSizeOfFirmDF = pd.read_csv('csv\ABSCS2018.AB1800CSA04-2023-04-20T164954.csv')

listOfDF = [YearinBusinessDF, ReceiptSizeOfFirmDF, EmploymentSizeOfFirmDF]

response = requests.get(url)
response

<Response [200]>

In [6]:
# The easy way to read in API data

data = response.json()

col = data[0]

df = pd.DataFrame(data[1:], columns=col)

colToDrop = ['GEO_ID', 'NAME', f'NAICS{year - 1}', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'PAYANN_F',
 'FIRMPDEMP_S', 'FIRMPDEMP_S_F', 'RCPPDEMP_S', 'RCPPDEMP_S_F', 'EMP_S', 'EMP_S_F', 'PAYANN_S',
 'PAYANN_S_F', 'us', 'YEAR', 'FIRMPDEMP_F', 'RCPPDEMP_F', 'EMP_F', 'VET_GROUP', 'EMPSZFI']

df = df.drop(columns= colToDrop, axis = 1).convert_dtypes().drop_duplicates()

# list(df.columns)
df.head()
# df.shape

,NAICS2017_LABEL,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP_LABEL,VET_GROUP_LABEL,EMPSZFI_LABEL,FIRMPDEMP,RCPPDEMP,EMP,PAYANN
0,Total for all sectors,Female,Hispanic,Total,Total,All firms,84496,91103725,722654,21975675
1,Total for all sectors,Female,Hispanic,Total,Total,Firms with no employees,10036,3233978,0,499996
2,Total for all sectors,Female,Hispanic,Total,Total,Firms with 1 to 4 employees,47105,15792524,92631,2808773
3,Total for all sectors,Female,Hispanic,Total,Total,Firms with 5 to 9 employees,12523,10688294,83449,2601817
4,Total for all sectors,Female,Hispanic,Total,Total,Firms with 10 to 19 employees,8516,14105603,116479,3506384


In [7]:
# Distinct column values

# df[['Column Name']].drop_duplicates()

,NAICS2017_LABEL
0,Total for all sectors
1144,"Agriculture, forestry, fishing and hunting"
1450,"Mining, quarrying, and oil and gas extraction"
2056,Manufacturing
2178,Utilities
2750,Construction
4754,Wholesale trade
6048,Retail trade
6844,Transportation and warehousing
7624,Information


Columns to keep from YearinBusinessDF:

 'Geographic Area Name (NAME)',
 '2017 NAICS code (NAICS2017)',
 'Meaning of NAICS code (NAICS2017_LABEL)',
 'Meaning of Sex code (SEX_LABEL)',
 'Meaning of Ethnicity code (ETH_GROUP_LABEL)',
 'Meaning of Race code (RACE_GROUP_LABEL)',
 'Meaning of Veteran code (VET_GROUP_LABEL)',
 'Meaning of Years in business code (YIBSZFI_LABEL)',
 'Number of employer firms (FIRMPDEMP)',
 'Sales, value of shipments, or revenue of employer firms ($1,000) (RCPPDEMP)',
 'Number of employees (EMP)',
 'Annual payroll ($1,000) (PAYANN)'

Columns to remove:

 'Year (YEAR)',
 'Relative standard error of employer firms (%) (FIRMPDEMP_S)',
 'Relative standard error of sales, value of shipments, or revenue of employer firms (%) (RCPPDEMP_S)',
 'Relative standard error of number of employees (%) (EMP_S)',
 'Relative standard error of annual payroll (%) (PAYANN_S)'
Column to merge from EmploymentSizeOfFirmDF:

- 'Meaning of Employment size of firms code (EMPSZFI_LABEL)'

 Column to merge from ReceiptSizeOfFirmDF:

- 'Meaning of Sales, value of shipments, or revenue size of firms code (RCPSZFI_LABEL)'

## Cleaning

- Drop columns we aren't planning to use
- Rename Columns we are planning to use
- Merge